In [6]:
from langchain.llms.openai import OpenAI  # 기본값으로 davinchi-003를사용
from langchain.chat_models import ChatOpenAI  # 기본값으로 GTP3.5-Turbo를 사용
from langchain.schema import HumanMessage, AIMessage, SystemMessage, BaseOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

# streaming=True, callbacks=[StreamingStdOutCallbackHandler()] 옵션을 넣어주면 실시간 출력을 볼 수 있다.
chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "country": "Japan",
        "answer": """
        I know this:
        Capital: Tokyo
        Languae: Japanese
        Food: Sushi
        :Currency: Yen
        """,
    }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])


example_prompt = FewShotChatMessagePromptTemplate(
    # ChatPromptTemplate을 사용해서 답변 양식을 만든다.
    example_prompt=example_prompt,
    # 실제 답변의 예시를 만든다.
    examples=examples,
)


final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert"),
    # 이전에 니가 이런식으로 답변했다는 것을 예로 들어준다.
    example_prompt,
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat

chain.invoke({
    "country":"Korea"
})

I know this:
Capital: Seoul
Language: Korean
Food: Kimchi
Currency: South Korean Won

AIMessageChunk(content='I know this:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi\nCurrency: South Korean Won')

In [ ]:
from langchain.schema import BaseOutputParser
# 받은 문자열을 ,을 기준으로 리스트로 만든다.
class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        # list 각 항목에 다시 str.strip 함수를 적용
        # str.strip 함수는 각 문자열의 앞 뒤 공백을 제거한다.
        return list(map(str.strip, items))

parser = CommaOutputParser()
    

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages(
    [("system", "You are a list generating machine. Everythig you are asked will be answered with a comma seperated list of max {max_items}. Do not reply with anything else."),
     ("human", "{question}"),]
)

# chain은 PromptTemplate | Model | OutputParser 로 만들 수 있다.
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "what are animals?"
})